***
# TEST
***

In [ ]:
import os
import json
import pandas as pd
import pvdeg
from pytest import approx
from pvdeg import TEST_DATA_DIR

# Load weather data
WEATHER = pd.read_csv(
    os.path.join(TEST_DATA_DIR, "weather_day_pytest.csv"), index_col=0, parse_dates=True
)
with open(os.path.join(TEST_DATA_DIR, "meta.json"), "r") as file:
    META = json.load(file)

# Load expected results
rh_expected = pd.read_csv(
    os.path.join(TEST_DATA_DIR, "input_day_pytest.csv"), index_col=0, parse_dates=True
)
rh_cols = [col for col in rh_expected.columns if "RH" in col]
rh_expected = rh_expected[rh_cols]


def test_module():
    """
    test pvdeg.humidity.calc_rel_humidity

    Requires:
    ---------
    weather dataframe and meta dictionary
    """
    result = pvdeg.humidity.module(WEATHER, META)
    pd.testing.assert_frame_equal(result, rh_expected, check_dtype=False)


def test_psat():
    """
    test pvdeg.humidity.psat

    Requires:
    ---------
    weahter dataframe and meta dictionary
    """
    psat_avg = pvdeg.humidity.psat(temp=WEATHER["temp_air"])[1]
    assert psat_avg == approx(0.47607, abs=5e-5)

In [ ]:
test_psat()

In [ ]:
import pvdeg
from pvdeg import TEST_DATA_DIR
import os
import pandas as pd

In [ ]:
sample_df = pd.read_csv(os.path.join(TEST_DATA_DIR, "diffusion_2d_input.csv"), index_col=0)

sample_df.index = pd.to_timedelta(sample_df.index)
sample_df.index = sample_df.index.values.astype("timedelta64[s]")

In [ ]:
res = pvdeg.diffusion.module_front(
    time_index=sample_df.index,
    backsheet_moisture=sample_df["Back Encapsulant Moisture"],
    sample_temperature=sample_df["Sample Temperature"],
    p=0.1,
    CW=15.6,
    nodes=20,
    eva_diffusivity_ea=0.395292897,
    Dif=2.31097881676966,
    n_steps=20,
)

In [ ]:
import pandas as pd
import numpy as np
import pvdeg
from pvdeg import TEST_DATA_DIR
import os

In [ ]:
setpoints = pd.read_csv(os.path.join(TEST_DATA_DIR, "chamber-setpoints.csv"), skiprows=[1])

In [ ]:
test_chamber = pvdeg.chamber.Chamber(os.path.join(TEST_DATA_DIR, "chamber-setpoints.csv"), setpoint_names=["temperature", "relative_humidity"], skiprows=[1])

test_chamber.setpoints = test_chamber.setpoints.iloc[:100]

Test Chamber Conditions

In [ ]:
chamber_result = test_chamber.chamber_conditions(tau_c=10, air_temp_0=25).iloc[:100]

chamber_result.to_csv("chamber_conditions_result.csv")

In [ ]:
CHAMBER_CONDITIONS = pd.read_csv("chamber_conditions_result.csv", index_col=0)
CHAMBER_CONDITIONS.index = CHAMBER_CONDITIONS.index.astype("timedelta64[s]")

pd.testing.assert_frame_equal(chamber_result, CHAMBER_CONDITIONS, check_dtype=False)

Test sample conditions

In [ ]:
test_chamber.setBacksheet(id='ST504', thickness=0.5) # PET
test_chamber.setEncapsulant(id='EVA', thickness=0.1) # EVA
sample_result = test_chamber.sample_conditions(tau_s=15, sample_temp_0=25, n_steps=20)

In [ ]:
sample_result.to_csv("sample_conditions_results.csv")

In [ ]:
SAMPLE_CONDITIONS = pd.read_csv(os.path.join(TEST_DATA_DIR, "sample_conditions_results.csv"), index_col=0)
SAMPLE_CONDITIONS.index = SAMPLE_CONDITIONS.index.astype("timedelta64[s]")

pd.testing.assert_frame_equal(sample_result, SAMPLE_CONDITIONS)

In [ ]:
test_chamber.plot_setpoints()

In [ ]:
test_chamber.calc_temperatures(air_temp_0=25, sample_temp_0=25, tau_c=10, tau_s=15)

pd.testing.assert_series_equal(test_chamber.air_temperature, CHAMBER_CONDITIONS["Air Temperature"], check_dtype=False)
pd.testing.assert_series_equal(test_chamber.sample_temperature, SAMPLE_CONDITIONS["Sample Temperature"], check_dtype=False)

In [ ]:
SAMPLE_CONDITIONS

In [ ]:
bad_df = pd.DataFrame(np.nan, index=pd.RangeIndex(5), columns=["Temperature"])

In [ ]:
pvdeg.chamber.setpoint_series(df=bad_df, setpoint_name="Temperature")

In [ ]:
import pytest
with pytest.raises(ValueError) as excinfo:
    pvdeg.chamber.setpoint_series(df=bad_df, setpoint_name="Temperature")

# Check that the exception message is correct
assert str(excinfo.value) == "column: Temperature contains NaN values. Remove from setpoint list or remove NaN's in input."

In [ ]:
import pandas as pd
import os 
import numpy as np
import pvdeg
from pvdeg import TEST_DATA_DIR

CHAMBER_CONDITIONS = pd.read_csv(
    os.path.join(TEST_DATA_DIR, "chamber_conditions_result.csv"), index_col=0
)
CHAMBER_CONDITIONS.index = CHAMBER_CONDITIONS.index.astype("timedelta64[s]")

SAMPLE_CONDITIONS = pd.read_csv(
    os.path.join(TEST_DATA_DIR, "sample_conditions_results.csv"), index_col=0
)
SAMPLE_CONDITIONS.index = SAMPLE_CONDITIONS.index.astype("timedelta64[s]")

    restored = Scenario.load_json(
        file_path=os.path.join(TEST_DATA_DIR, "test-scenario.json")
    )

    a.path, restored.path = None, None
    a.file, restored.file = None, None

    assert a == restored

In [1]:
import pvdeg

In [ ]:
pvdeg.utilities.pvdeg_datafiles

In [19]:
def test_read_material_special():

    template_material = pvdeg.utilities.read_material(pvdeg_file="AApermeation", key="AA000")

    assert len(template_material) == 1
    assert "comment" in template_material

test_read_material_special()

In [ ]:
template_material = pvdeg.utilities.read_material(pvdeg_file="AApermeation", key="AA000")

[type(x) for x in template_material.values()][0]

In [21]:
def test_read_material_normal():

    res = {
        'name': 'ST504', 
        'alias': 'PET1', 
        'contributor': 'Michael Kempe', 
        'source': 'unpublished measurements', 
        'Fickian': True,
        'Ead': 47.603, 
        'Do': 0.554153, 
        'Eas': -11.5918, 
        'So': 9.554366e-07, 
        'Eap': 34.2011, 
        'Po': 2128.8937
    }

    template_material = pvdeg.utilities.read_material(pvdeg_file="O2permeation", key="OX002")

    assert template_material == res

test_read_material_normal()

In [22]:
def test_read_material_fewer_params():

    res = {
        'name': 'ST504', 
        'Fickian': True,
    }

    template_material = pvdeg.utilities.read_material(pvdeg_file="O2permeation", key="OX002", parameters=["name", "Fickian"])

    assert template_material == res

test_read_material_fewer_params()

In [24]:
def test_read_material_extra_params():

    res = {
        'namenotindict1': None,
        'namenotindict2': None,
    }

    template_material = pvdeg.utilities.read_material(pvdeg_file="O2permeation", key="OX002", parameters=["namenotindict1", "namenotindict2"])

    assert template_material == res


test_read_material_extra_params()

In [ ]:
def test_search_json():


In [5]:
# pvdeg_file should override fp if both are provided
def test_read_material_fp_override():

    res = {
        'name': 'ST504', 
        'alias': 'PET1', 
        'contributor': 'Michael Kempe', 
        'source': 'unpublished measurements', 
        'Fickian': True,
        'Ead': 47.603, 
        'Do': 0.554153, 
        'Eas': -11.5918, 
        'So': 9.554366e-07, 
        'Eap': 34.2011, 
        'Po': 2128.8937
    }

    from pvdeg import DATA_DIR

    # pass pvdeg file and it gets overridden by the file path
    template_material = pvdeg.utilities.read_material(
        pvdeg_file="O2permeation", 
        fp=os.path.join(DATA_DIR, "AApermeation.json"), 
        key="OX002",
    )

    assert template_material == res

test_read_material_fp_override()

In [9]:
def test_search_json():

    name_res = pvdeg.utilities.search_json(pvdeg_file="H2Opermeation", name_or_alias="Ethylene Vinyl Acetate")
    alias_res = pvdeg.utilities.search_json(pvdeg_file="H2Opermeation", name_or_alias="EVA")

    assert name_res == "W001"
    assert alias_res == "W001"

test_search_json()

In [ ]:
from pvdeg import DATA_DIR

pvdeg_file = "H2Opermeation"
invalid_name_or_alias = "namenotindict"
expected_error_message = (
    rf"name_or_alias: {invalid_name_or_alias} not in JSON at "
    rf"{os.path.join(DATA_DIR, pvdeg.utilities.pvdeg_datafiles[pvdeg_file])}"
)

test_chamber.setDimensions()

test_chamber.setpoints = test_chamber.setpoints.iloc[:100]  # we only care about the first 100 setpoints

In [ ]:
test_chamber

In [ ]:
expected_error_message

In [ ]:
pvdeg.utilities.search_json(pvdeg_file=pvdeg_file, name_or_alias=invalid_name_or_alias)

In [10]:
import os
import pandas as pd
import pvdeg
import numpy as np
from pvdeg import TEST_DATA_DIR
import pytest

INPUT_SPECTRA = os.path.join(TEST_DATA_DIR, r"spectra_pytest.csv")

data = pd.read_csv(INPUT_SPECTRA)
wavelengths = np.array(range(280, 420, 20))

# convert to expected format
spectra = data["Spectra"]
spectra_df = pd.DataFrame(spectra.tolist(), index=spectra.index)
spectra_df = spectra.str.strip("[]").str.split(",", expand=True).astype(float)
spectra_df.columns = wavelengths

# from input data, this was lost during our original conversion to the dataframe
spectra_df.index = pd.date_range("2021-03-09 10:00:00", freq='1h', periods=10)

conditions_df = pd.DataFrame(
    data={
        "relative_humidity": data["RH"],
        "temperature": data["Temperature"],
    }
)
conditions_df.index = spectra_df.index

degradation = pvdeg.degradation.degradation(
    spectra_df=spectra_df,
    conditions_df=conditions_df
)

assert degradation == pytest.approx(4.4969e-38, abs=0.02e-38)

In [ ]:
degradation